In [96]:
hydra.core.global_hydra.GlobalHydra().instance().clear()

In [97]:
hydra._internal.GlobalHydra().clear()

AttributeError: module 'hydra._internal' has no attribute 'GlobalHydra'

In [98]:
hydra._internal.GlobalHydra().get_state().clear()

AttributeError: module 'hydra._internal' has no attribute 'GlobalHydra'

In [99]:
import hydra
from hydra.experimental import initialize, compose
import numpy as np
import tqdm
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer
from omegaconf import DictConfig

from torchfly.text.decode import TransformerDecoder
from torchfly.common import set_random_seed, move_to_device

from configure_dataloader import DataLoaderHandler
from model import Generator, TextGAILModel

import logging
import os
import pathlib

In [29]:
random_seed = 1
set_random_seed(random_seed)

1

In [62]:
hydra.experimental.initialize("config")

hydra.experimental.initialize()

In [104]:
# config = hydra.experimental.compose("config.yaml")
hydra.core.global_hydra.GlobalHydra().instance().clear()

with initialize(config_path='config'):
  config = compose(config_name="config")
# cfg = hydra.experimental.compose(config_file='config.yaml', overrides=[])
print(config.pretty())

/home/aigo/anaconda3/envs/torchfly/lib/python3.7/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive task/CommonGEN.yaml in file:///home/aigo/TextGAIL/Conditional/Evaluation/config.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarning)
/home/aigo/anaconda3/envs/torchfly/lib/python3.7/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive training/CommonGEN.yaml in file:///home/aigo/TextGAIL/Conditional/Evaluation/config.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarning)
/home/aigo/anaconda3/envs/torchfly/lib/python3.7/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive model/roberta-tokenized-gpt2.yaml in file:///home/aigo/TextGAIL/Conditional/Evaluation/config.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_

task:
  name: CommonGEN
  data_dir: ../../../../data/CommonGEN
  mle_weights_path: ''
  textgail_weights_path: ''
training:
  random_seed: 1
  num_gpus_per_node: 1
  batch_size: 32
  resume:
    resume: false
    resume_model: true
    resume_optimizer: true
    resume_scheduler: true
    resume_rng_state: true
  total_num:
    epochs: -1
    update_steps: 1000
  validation:
    steps_interval: 100
  checkpointing:
    directory: Checkpoints
    steps_interval: 100
    seconds_interval: -1
    num_checkpoints_to_keep: 1000
    keep_checkpoint_every_num_seconds: 86400
  logging:
    level: INFO
    steps_interval: -1
    seconds_interval: 2
  optimization:
    fp16: false
    fp16_opt_level: O1
    optimizer_name: AdamW
    learning_rate: 1.0e-05
    gradient_accumulation_steps: 1
    weight_decay: 0.01
    max_gradient_norm: -1.0
    warmup:
      scheduler_name: WarmupLinear
      warmup_steps: 100
model:
  initializer_range: 0.02
  layer_norm_epsilon: 1.0e-05
  n_ctx: 1024
  n_embd: 

In [105]:
config

{'task': {'name': 'CommonGEN', 'data_dir': '../../../../data/CommonGEN', 'mle_weights_path': '', 'textgail_weights_path': ''}, 'training': {'random_seed': 1, 'num_gpus_per_node': 1, 'batch_size': 32, 'resume': {'resume': False, 'resume_model': True, 'resume_optimizer': True, 'resume_scheduler': True, 'resume_rng_state': True}, 'total_num': {'epochs': -1, 'update_steps': 1000}, 'validation': {'steps_interval': 100}, 'checkpointing': {'directory': 'Checkpoints', 'steps_interval': 100, 'seconds_interval': -1, 'num_checkpoints_to_keep': 1000, 'keep_checkpoint_every_num_seconds': 86400}, 'logging': {'level': 'INFO', 'steps_interval': -1, 'seconds_interval': 2}, 'optimization': {'fp16': False, 'fp16_opt_level': 'O1', 'optimizer_name': 'AdamW', 'learning_rate': 1e-05, 'gradient_accumulation_steps': 1, 'weight_decay': 0.01, 'max_gradient_norm': -1.0, 'warmup': {'scheduler_name': 'WarmupLinear', 'warmup_steps': 100}}}, 'model': {'initializer_range': 0.02, 'layer_norm_epsilon': 1e-05, 'n_ctx': 1

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
dataloader_handler = DataLoaderHandler(config)
valid_dataloader = dataloader_handler.test_dataloader(config)
collate_fn = test_dataloader.dataset.collate_fn

In [ ]:
device = torch.device("cuda")

In [ ]:
model = TextGAILModel(config)
model = model.cuda()

In [ ]:
decoder = TransformerDecoder(config.decode)
decoder.register_generator(model.generator.decoder)
decoder.register_tokenizer(tokenizer)
decoder.prepare_model_inputs_for_generation = model.generator.prepare_model_inputs_for_generation

## MLE Generation

In [ ]:
# location to store results
os.makedirs(config.task.name,exist_ok=True)

In [ ]:
print(config.task.mle_weights_path)

In [ ]:
mle_weights = torch.load(config.task.mle_weights_path)
model.generator.load_state_dict(mle_weights)

In [ ]:
temperatures = (np.arange(10) + 1) / 10.0

In [ ]:
for temperature in temperatures:
    f_write = open(f"{config.task.name}/mle_{temperature}_{random_seed}.txt", "w")

    for batch in tqdm.tqdm(valid_dataloader):
        batch = collate_fn(batch)
        batch = move_to_device(batch, device)

        ground_truth = batch["target_text"]

        results = decoder.generate(batch["source_token_ids"], temperature=temperature)
        generated = []

        for i in range(len(results["tokens"])):
            res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
            generated.append(res)

        for gt, gen in zip(ground_truth, generated):
            f_write.write(json.dumps([gt, gen]))
            f_write.write("\n")
        
    f_write.close()

### Beam Search

In [ ]:
# f_write = open(f"{config.task.name}/mle_beam_4.txt", "w")

# for batch in tqdm.tqdm(valid_dataloader):
#     batch = collate_fn(batch)
#     batch = move_to_device(batch, device)

#     ground_truth = batch["target_text"]

#     results = decoder.generate(batch["source_token_ids"], do_sample=False, num_beams=4)
#     generated = []

#     for i in range(len(results["tokens"])):
#         res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
#         generated.append(res)

#     for gt, gen in zip(ground_truth, generated):
#         f_write.write(json.dumps([gt, gen]))
#         f_write.write("\n")        

# f_write.close()

## TextGAIL Generation

In [ ]:
print(config.task.textgail_weights_path)

In [ ]:
textgail_weights = torch.load(config.task.textgail_weights_path)
model.load_state_dict(textgail_weights)

In [ ]:
for temperature in temperatures:
    f_write = open(f"{config.task.name}/textgail_{temperature}_{random_seed}.txt", "w")

    for batch in tqdm.tqdm(valid_dataloader):
        batch = collate_fn(batch)
        batch = move_to_device(batch, device)

        ground_truth = batch["target_text"]

        results = decoder.generate(batch["source_token_ids"], temperature=temperature)
        generated = []

        for i in range(len(results["tokens"])):
            res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
            generated.append(res)

        for gt, gen in zip(ground_truth, generated):
            f_write.write(json.dumps([gt, gen]))
            f_write.write("\n")
        
    f_write.close()

### Beam Search

In [ ]:
f_write = open(f"{config.task.name}/textgail_no_pretrain2_beam_4.txt", "w")

for batch in tqdm.tqdm(valid_dataloader):
    batch = collate_fn(batch)
    batch = move_to_device(batch, device)

    ground_truth = batch["target_text"]

    results = decoder.generate(batch["source_token_ids"], do_sample=False, num_beams=4)
    generated = []

    for i in range(len(results["tokens"])):
        res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
        generated.append(res)
        
    for gt, gen in zip(ground_truth, generated):
        f_write.write(json.dumps([gt, gen]))
        f_write.write("\n")        

f_write.close()